In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Model loading

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloomz-3b",
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-3b")

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloomz-3b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model Architecture

It's important to observe the model's construction so you can ensure you know which modules you should apply LoRA to.

As per the paper, we're going to focus on the attention weights - so keep an eye out for modules like: q_proj, v_proj, query_key_value. This is model dependet

In [ ]:
print(model)

BloomForSequenceClassification(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2560)
    (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear8bitLt(in_features=2560, out_features=7680, bias=True)
          (dense): Linear8bitLt(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear8bitLt(in_features=2560, out_features=10240, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear8bitLt(in_features=10240, out_features=2560, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2560,), eps

 Post-processing on the model

Finally, we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in float32 for stability. We also cast the output of the last layer in float32 for the same reasons.

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

Preprocessing

We can simply load our dataset from 🤗 Hugging Face with the load_dataset method!

In [ ]:
import transformers
from datasets import load_dataset

dataset_name = "FourthBrainGenAI/MarketMail-AI"
product_name = "product"
product_desc = "description"
product_ad = "marketing_email"

In [ ]:
dataset = load_dataset(dataset_name)
print(dataset)
print(dataset['train'][0])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/17 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/FourthBrainGenAI___parquet/FourthBrainGenAI--MarketMail-AI-2490edbbc6c1c289/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['product', 'description', 'marketing_email'],
        num_rows: 17
    })
})
{'product': 'SmartEyes', 'description': 'Glasses with real-time translation', 'marketing_email': "Subject Line: 👓 Unleash Multilingual Magic with SmartEyes Glasses - See the World in YOUR Language! 🌍\n\nHey there, Language Lover! 🌟\n\nDo you dream of seamlessly navigating through conversations in any language? 🙌 Guess what! Your linguistic aspirations are about to come true with our *brand new* product - SmartEyes Glasses! 😎\n\nPicture this - you're strolling through the bustling streets of Tokyo 🇯🇵, admiring the stunning sakura blossoms. Suddenly, a friendly local strikes up a conversation. Worry not! Your SmartEyes Glasses are here to help! 🌸✨\n\nWhy choose SmartEyes, you ask? Here's the scoop: \n\n🎯 Real-time Translation: With the cutting-edge technology packed into SmartEyes Glasses, experience lightning-fast translations, making exchanges smooth and na

dataform

In [ ]:
def generate_prompt(product: str, description: str, marketing_email: str) -> str:
  prompt = f"### INSTRUCTION\nBelow is a product and description, please write a marketing email for this product.\n\n### Product:\n{product}\n### Description:\n{description}\n\n### Marketing Email:\n{marketing_email}"
  return prompt

mapped_dataset = dataset.map(lambda samples: tokenizer(generate_prompt(samples['product'], samples['description'], samples['marketing_email'])))

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=mapped_dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=6,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=100,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,2.061600
2,0.684400
3,1.378700
4,1.370800
5,0.676800
6,2.026700
7,2.015200
8,0.681800
9,1.304900
10,1.299100


TrainOutput(global_step=100, training_loss=0.5406914208317175, metrics={'train_runtime': 400.4278, 'train_samples_per_second': 5.994, 'train_steps_per_second': 0.25, 'total_flos': 8697930396211200.0, 'train_loss': 0.5406914208317175, 'epoch': 66.67})

Share adapters on the 🤗 Hub



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

If you run into issues during upload - please ensure you're using a HF Token with WRITE access!

In [ ]:
model.push_to_hub("rohit/email_gen", use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/FourthBrainGenAI/BLOOMZ-3b-marketmail-ai/commit/dd84fbbb06c121e2f0f4412e49a0c2afd1b2b476', commit_message='Upload model', commit_description='', oid='dd84fbbb06c121e2f0f4412e49a0c2afd1b2b476', pr_url=None, pr_revision=None, pr_num=None)

Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "rohit/email_gen"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Inference

Take it for a spin!

In [ ]:
from IPython.display import display, Markdown

def make_inference(product, description):
  batch = tokenizer(f"### INSTRUCTION\nBelow is a product and description, please write a marketing email for this product.\n\n### Product:\n{product}\n### Description:\n{description}\n\n### Marketing Email:\n", return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=200)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [ ]:
your_product_name_here = "The Coolinator"
your_product_description_here = "A personal cooling device to keep you from getting overheated on a hot summer's day!"

make_inference(your_product_name_here, your_product_description_here)

### INSTRUCTION
Below is a product and description, please write a marketing email for this product.

### Product:
The Coolinator
### Description:
A personal cooling device to keep you from getting overheated on a hot summer's day!

### Marketing Email:
Subject Line: 🌞🌞💥 Go HOT with The Coolinator - Your Ultimate Cooling Journey! 🌬🌊

Hey there, Cool Minded Friend! 🌊🌊

Do you ever feel like your head just swings off on a hot day? Do you wish you could cool down your body and your brain right there and then? For those who love to exercise, the Coolinator is the perfect gadget to keep you cool and motivated! 💪💪

Introducing the Coolinator - your personal cooling device that helps you stay as cool as a breeze! 💦💦

🌟 Unleash your inner Steve Jobs and introduce the Coolinator to your friends and family so they too can cool off! 🌬🌊

🌟 Head to the gym, the park, or just about anywhere else your body needs a cool breather! 🌬�